In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [4]:
import logging
import pandas as pd
import awswrangler as wr
from matplotlib import pyplot as plt
import seaborn as sns

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId

In [3]:
customer = "Tchibo"
start_date = "20250902"
end_date = "20250903"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [7]:
audiences = call_api_with_accountId(
    endpoint_url=f"{url}/audiences/query", accountID=account_id, content={"status":"active"}, logger=logging
)
audiences

In [9]:
res = pd.DataFrame(columns=["id","audience_name","targeting_date","count"])
for audience in audiences:
    audience_id = audience["id"]
    audience_name = audience["name"]
    print(f"audience_name = {audience_name}")
    for date in pd.date_range(start=start_date, end=end_date, freq="D"):
        date = date.strftime("%Y%m%d")
        try:
            df = wr.s3.read_parquet(f"s3://{account_id}/targeting.history/{date}/{audience_id}.parquet")
            df = df[df["treatment"]==audience_id]
            count = df.shape[0]
        except wr.exceptions.NoFilesFound:
            print(f"No files found for audience {audience_name} on date {date}")
            count = 0
        res.loc[len(res)] = [audience_id,audience_name,date,count]
res

In [13]:
res_sum = res.groupby(by=["id","audience_name"])["count"].sum().reset_index()
res_sum